In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os
import re

import pickle

from etcpy import etc
from etcpy import tempDep

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [2]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params_new_tagged.csv'),index_col=0)
df = etc.calculate_thermal_params(params)

dfae_batch = data[0].set_index('Ts').rename_axis(None)
params

,Topt,Topt_std,Length,Tm,Tm_std,T90,dCpt,dCpt_std,topt_source,TmTag
P77467,306.15,10,262,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76077,311.15,10,309,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76078,310.15,10,95,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
P76079,309.15,10,248,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76081,310.15,10,356,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
...,...,...,...,...,...,...,...,...,...,...
P0AG80,313.15,10,438,329.166303,4.070000,330.45,-4000,1000,predicted,Exp
P77718,307.15,10,482,319.125751,4.070000,320.15,-4000,1000,predicted,Exp
P0A6D5,308.15,10,288,328.762604,7.710000,NaN,-4000,1000,predicted,Mean
standard,308.15,10,272,328.762604,7.710000,NaN,-4000,1000,BullShit,Mean


In [3]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('../models/ecModel.mat')
    
    match = re.compile('usage_prot_*', re.IGNORECASE)
    # for react in mae.reactions:
    #     if match.match(react.id) and react.lower_bound != -1000:
    #         ori_lb = react.lower_bound
    #         react.lower_bound = ori_lb*2.5


    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = 50
    #mae.medium['EX_glc__D_e'] = 100
    
    try: 
        rae, gae, oae, cae, aae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: 
        rae = np.zeros(dfae_batch.shape[0])
        model = None
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    rexp = dfae_batch['r'].values
    #print(rae)
    print(abs(rae - dfae_batch['r'].values))
    print('r2_batch:',r2_score(rexp,rae))
    print('MSE_ae',MSE(rexp,rae))
    return {'growth':np.array(rae), 'glucose':np.array(gae), 'oxygen': np.array(oae), 'carbonDioxide': np.array(cae), 'acetate': np.array(aae)}, r2_score(rexp,rae), MSE(rexp,rae)

In [4]:
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])

for i in range(1,11):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_random_topt(params, ['Topt'])
    rae, score, mse = aerobic(newParams)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-06
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.29211640702503106
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.3796229136163389
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.48112384679161513
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.5356553354102669
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5799737737825059
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.6084352978677131
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.5708184919442116
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.5031199211496663
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  0.4522483984664685
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  0.40858307617178125
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  0.35308188466727897
NGAM is: 6.9000236503978725
Growth at  44.45349

In [5]:
df.to_csv('../results/wigTopt.csv')

In [6]:
df

,iter,temp,growth,o2flux,gluFlux,co2flux,acetateFlux,R2,MSE
0,1,20.153299,0.292116,8.948315,7.551705,-7.561528,-0.811672,-1.872085,0.442018
1,1,23.001511,0.379623,11.185749,8.521314,-9.988729,-0.734343,-1.872085,0.442018
2,1,26.331896,0.481124,13.559607,9.380210,-12.828594,-1.304250,-1.872085,0.442018
3,1,28.251820,0.535655,14.427916,8.996895,-14.531334,-2.492089,-1.872085,0.442018
4,1,29.994212,0.579974,15.189800,9.012437,-15.865407,-3.828450,-1.872085,0.442018
...,...,...,...,...,...,...,...,...,...
10,10,42.238764,0.356206,10.470860,4.721382,-11.230930,-1.237214,-1.835993,0.436463
11,10,44.453491,0.292768,9.026056,3.993261,-9.650763,-1.145051,-1.835993,0.436463
12,10,45.289899,0.270414,8.517728,3.743581,-9.094737,-1.127218,-1.835993,0.436463
13,10,46.645331,0.236306,7.727576,3.359148,-8.231804,-1.106212,-1.835993,0.436463
